# 魔円陣をサイズが小さいものから列挙する
#### **Julia** を用いる。**Pkg** は, **Nemo** を用いる。

In [1]:
println(versioninfo())
using Nemo,Combinatorics,Dates,Primes

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M3
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
nothing

Welcome to Nemo version building

Nemo comes with absolutely no warranty whatsoever


### 先ずシンプルに位数が素数$p$である有限体$GF(p)$の3次拡大から有限体を構成する作業を関数化する

In [2]:
function makeMagicCircleS(p)
    q=p^2+p+1
    pp=p^3-1
    Fp=GF(p)
#    println(p,",",q,", size = ",length(Fp))
#    println(join(Fp,", "))
    Fpx, x = Fp["x"]
    IPlst=[]
    for a ∈ Fp, b ∈ Fp, c ∈ Fp
        f(x)=x^3+a*x^2+b*x+c
        chk=true
        for e ∈ Fp
            if f(e)==0
                chk=false
                break
            end
        end
        if chk
            push!(IPlst,f(x))
        end
    end
#    if length(IPlst)>20
#        println(join(IPlst[1:10],", "),"...",join(IPlst[end-9:end],", "))
#    else
#        println(join(IPlst,", "))
#    end
#    println("既約多項式の候補の数 = ",length(IPlst))
### 原始元のチェックのために, p^3-1の約数の配列を準備する。
    sqrpp=Int(floor(sqrt(pp)))
#    println(pp,",",sqrpp)
    pf=[]
    for i=1:sqrpp
        if rem(pp,i)==0
            push!(pf,i)
            push!(pf,Int(pp/i))
        end
    end
    sort!(unique!(pf))
    pop!(pf)
    popfirst!(pf)
#    println(join(pf,","))
### いよいよ魔円陣
    Fpt=[]
    Fptl=[]
    α=0
    chk=false
    r=0
    for k=1:length(IPlst)
        Fpt,α=finite_field(IPlst[k],"α")
        for r=0:0#p-1
#        Fptl=[]
#        push!(Fptl,1)
#        for i=1:length(Fpt)
            chkA=true
            for i ∈ pf
                e=(α+r)^i
                if e==1
                    chkA=false
                    break
#            else
#                push!(Fptl,e)
                end
            end
            if chkA
#        println(k," r=",r," : size = ",length(Fptl), ", p^3-1 = ",p^3-1,", ",IPlst[k])
#        if length(unique(Fptl))==p^3-1
#                println(Fpt)
#                println(k,", r=",r," : ",IPlst[k])
                chk=true
                break
            end
        end
        if chk
            break
        end
    end
#println(join(Fptl[1:10],", "),"...",join(Fptl[end-9:end],", "))
    pdsl=[0]
    for i=1:length(Fpt)
        if coeff((α+r)^i,2)==0
#        println(i,": ",α^i)
            push!(pdsl,i)
        end
    end
#    println(pdsl[1:10],"...",pdsl[end-9:end])
    bbl=[]
    for i=1:p+1
        push!(bbl,pdsl[i+1]-pdsl[i])
    end
#    println("(",join(bbl,","),")")
### できた魔円陣の配列bblから他の同サイズの魔円陣を列挙する
    cds=[0]
    for i=1:length(bbl)
        push!(cds,cds[end]+bbl[i])
    end
    #println("Perfect Difference Set = (",join(cds,","),")")
    σ=p^2+p+1
#    println("σ = ",σ)
    otherlst=[]
    for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
        cdsa=sort!((((cds.*i).-1).%σ).+1)
        push!(otherlst,cdsa)
    end
    oblst=[]
    for lst ∈ otherlst
        obll=[]
        for i=2:length(lst)
            push!(obll,lst[i]-lst[i-1])
        end
#    sort!(obll)
        while obll[1]>1
            circshift!(obll,-1)
        end
        if obll[2]>obll[end]
            obll=[obll[1];reverse(obll[2:end])]
        end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
        if !(obll ∈ oblst) && !(0 ∈ obll)
            push!(oblst,obll)
        end
    end
#    println("同じサイズの魔円陣の数は, ",length(oblst))
    sort!(oblst, lt= (a,b)->a[2]*10+a[3]<b[2]*10+b[3])
    ansl=[]
    for bll ∈ oblst
#        println("(",join(bll,","),")")
        push!(ansl,bll)
    end
    ansl
end

makeMagicCircleS (generic function with 1 method)

In [3]:
bbll=makeMagicCircleS(5)
plnstrlst=[]
for bbl ∈ bbll
    push!(plnstrlst,"("*join(bbl,",")*")")
end
println(join(plnstrlst,", "))

(1,2,5,4,6,13), (1,2,7,4,12,5), (1,3,2,7,8,10), (1,3,6,2,5,14), (1,7,3,2,4,14)


### 次に, 位数が素数$p$の累乗$n>1$である有限体$GF(p^n)$の3次拡大から有限体を構成する作業を関数化する

In [4]:
function makeMagicCircleC(p,n)
    q=p^n
    qq=q^3-1
#    println("p = ",p,", n = ",n,", q=p^n= ",q)
    Fp,α=finite_field(p,n,"α")
#    for e ∈ Fp
#        print(e,", ")
#    end
#    print("\n")
#    println("Fpのサイズは, ",length(Fp))
    start = Dates.now()
### 関数環を用意して3次拡大する
    Fpx, x = Fp["x"]
    IPlst=[]
    for a ∈ Fp, b ∈ Fp, c ∈ Fp
        f(x)=x^3+a*x^2+b*x+c
        chk=true
        for e ∈ Fp
            if f(e)==0
                chk=false
                break
            end
        end
        if chk && (a!=0 || b!=0) # ちょっと注文付けてみた... 
            push!(IPlst,f(x))
        end
#    if length(IPlst)>2000
        if Dates.now() - start > Millisecond(2000)
            break
        end
    end
#    println(length(IPlst))
#    println(join(IPlst[1:10],",")," ... ",join(IPlst[end-9:end],","))
    dtime = Dates.now() - start
#    println("かかった時間: $dtime")
#
    sqrqq=Int(floor(sqrt(qq)))
#    println(p,", ",n,", ",q,", ",qq,", ",sqrqq)
    qf=[]
    for i=1:sqrqq
        if rem(qq,i)==0
            push!(qf,i)
            push!(qf,Int(qq/i))
        end
    end
    sort!(unique!(qf))
    pop!(qf)
    popfirst!(qf)
#    println(qq," の約数は, ",join(qf,","))
    start = Dates.now()
### 魔円陣を拵える
    ω=0
    r=0
    chk=false
    ωω=ω
    for k=1:length(IPlst)
        Fpt,ω=finite_field(IPlst[k],"ω")
        for r=0:p-1
            ωω=ω+r
            Fptl=[]
            push!(Fptl,1)
#        for i=1:length(Fpt)
            chkA=true
            i=0
            for i ∈ qf
#    e=ω^i
                e=ωω^i
#    println(i,": ",e)
                if e==1
                    chkA=false
                    break
#            else
#                push!(Fptl,e)
                end
            end
            dtime = Dates.now() - start
#            println(k,", ",ωω," : i = ",i, ", p^3-1 = ",(p^n)^3-1,", かかった時間: $dtime millis")
#        if length(Fptl)==(p^n)^3-1
            if chkA
#            println(Fpt)
#                println(k,", r=",r," : ",IPlst[k])
                chk=true
                break
            end
        end
        if chk
            break
        end
    end
#println(join(Fptl[1:10],", ")," ... ",join(Fptl[end-9:end],", "))
    dtime = Dates.now() - start
#    println("かかった時間: $dtime")
    pdsl=[0]
    for i=1:qq
        if coeff(ωω^i,2)==0
#    if coeff(ω^i,2)==0
#        println(i,": ",α^i)
            push!(pdsl,i)
        end
    end
#    println(join(pdsl[1:10],","),"...",join(pdsl[end-9:end],","))
    bbl=[]
    for i=1:p^n+1
        push!(bbl,pdsl[i+1]-pdsl[i])
    end
#    println(join(bbl,","))
#    println("size = ",length(bbl))
#    dtime = Dates.now() - start
### チェックを掛ける    
    start = Dates.now()
    bblw=[bbl;bbl]
#println(bblw)
    suml=[]
    for b=1:length(bbl)-1
        for i=1:length(bbl)
            sum=0
            for j=i:i+b-1
                sum=sum+bblw[j]
            end
            push!(suml,sum)
        end
    end
    push!(suml,sum(bbl))
#    println(join(sort(unique(suml))[1:20],",")," ... ",join(sort(unique(suml))[end-10:end],",")," : ",length(suml))
    dtime = Dates.now() - start
#    println("かかった時間: $dtime")
### 同じサイズの魔円陣を列挙する
    cds=[0]
    for i=1:length(bbl)
        push!(cds,cds[end]+bbl[i])
    end
#    println("Perfect Difference Set = (",join(cds,","),")")
    σ=(p^n)^2+(p^n)+1
#    println("σ = ",σ)
    otherlst=[]
    for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
        cdsa=sort!((((cds.*i).-1).%σ).+1)
        push!(otherlst,cdsa)
    end
    oblst=[]
    for lst ∈ otherlst
        obll=[]
        for i=2:length(lst)
            push!(obll,lst[i]-lst[i-1])
        end
#    sort!(obll)
        while obll[1]>1
            circshift!(obll,-1)
        end
        if obll[2]>obll[end]
            obll=[obll[1];reverse(obll[2:end])]
        end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
        if !(obll ∈ oblst) && !(0 ∈ obll)
            push!(oblst,obll)
        end
    end
#    println("同じサイズ ",length(oblst[1])," な魔円陣の数は, ",length(oblst))
    sort!(oblst, lt= (a,b)->a[2]*10+a[3]<b[2]*10+b[3])
#    if length(oblst)>20
#        for bll ∈ oblst[1:10]
#            println("(",join(bll,","),")")
#        end
#        println("...")
#        for bll ∈ oblst[end-9:end]
#            println("(",join(bll,","),")")
#        end
#    end
    oblst
end

makeMagicCircleC (generic function with 1 method)

In [5]:
bbll=makeMagicCircleC(5,2)
plnstrlst=[]
for bbl ∈ bbll
    push!(plnstrlst,"("*join(bbl,",")*")")
end
println(join(plnstrlst,", "))

(1,3,11,55,19,33,10,17,12,81,90,13,8,28,35,2,22,18,38,26,32,44,7,16,25,5), (1,3,14,10,51,26,5,15,70,13,35,2,36,7,9,60,72,12,53,6,56,32,23,11,8,21), (1,2,30,12,13,22,28,6,5,10,38,14,26,64,8,19,4,20,17,29,36,18,76,9,7,137), (1,2,40,21,9,19,69,8,6,39,37,17,41,87,25,32,18,29,5,10,12,4,7,13,35,65), (1,5,25,19,4,35,29,12,10,16,2,15,42,36,20,7,65,9,62,8,3,21,13,132,14,46), (1,7,9,10,90,59,60,4,57,11,2,18,36,22,15,6,23,5,35,12,30,3,50,48,14,24), (1,7,12,44,42,30,49,5,24,2,9,60,23,39,76,36,25,21,6,16,59,15,3,10,4,33), (1,8,5,6,32,2,22,3,23,7,66,17,41,36,10,18,21,42,29,15,103,12,4,31,37,60), (1,3,57,26,24,44,59,54,12,10,69,23,7,9,31,25,2,62,11,42,29,6,8,5,15,17), (1,8,12,53,51,11,6,26,5,2,22,16,14,4,23,19,84,25,10,82,15,13,36,44,3,66), (1,6,45,20,53,14,8,16,26,5,30,9,4,15,18,3,57,23,96,32,2,25,29,12,92,10), (1,9,18,11,22,20,6,15,70,8,5,45,90,52,3,14,40,25,24,7,12,4,84,30,2,34), (1,9,20,24,8,14,3,23,51,19,15,82,33,2,4,80,42,13,5,38,7,64,36,31,16,11), (1,9,23,25,4,18,71,30,6,13,7,8,31,35,2,14,24,3

### 小さい方から順に魔円陣を列挙していく

In [6]:
tstart = Dates.now()
pgnlst=[]
max=128
#p=BigInt(1)
for p=2:400,n=1:20
    if isprime(p) && p^n<=max && p^n>0
        q=p^n
        push!(pgnlst,[p,n,q,q^2+q+1,q+1,q^3+1])
    end
end
sort!(pgnlst,by = x->x[5])
g=1
for pgn ∈ pgnlst
    if pgn[5]<=400
#        println(g," : ",join(pgn,","))
        g=g+1
        if pgn[2]==1
            bbll=makeMagicCircleS(pgn[1])
        else
            bbll=makeMagicCircleC(pgn[1],pgn[2])
        end
        println("p=",pgn[1],", n=",pgn[2],", q=",pgn[3],", q^2+q+1=",pgn[4],", q+1=",pgn[5],", q^3+1=",pgn[6],", Number of MagicCircles = ",length(bbll))
        plnstrlst=[]
        if length(bbll)<=6
            for bbl ∈ bbll
                push!(plnstrlst,"("*join(bbl,",")*")")
            end
            println(join(plnstrlst,", "))
        else
            for bbl ∈ bbll[1:3]
                push!(plnstrlst,"("*join(bbl,",")*")\n")
            end
            println(join(plnstrlst,", ")," ... ")
            plnstrlst=[]
            for bbl ∈ bbll[end-2:end-1]
                push!(plnstrlst,"("*join(bbl,",")*")\n")
            end
            push!(plnstrlst,"("*join(bbll[end],",")*")")
            println(join(plnstrlst,", "))
        end
    end    
    dtime = Dates.now() - tstart
    println("かかった時間(途中経過): $dtime")
    println("")
end
dtime = Dates.now() - tstart
println("fin.  かかった時間: $dtime")
    

p=2, n=1, q=2, q^2+q+1=7, q+1=3, q^3+1=9, Number of MagicCircles = 1
(1,2,4)
かかった時間(途中経過): 156 milliseconds

p=3, n=1, q=3, q^2+q+1=13, q+1=4, q^3+1=28, Number of MagicCircles = 2
(1,2,6,4), (1,3,2,7)
かかった時間(途中経過): 165 milliseconds

p=2, n=2, q=4, q^2+q+1=21, q+1=5, q^3+1=65, Number of MagicCircles = 1
(1,3,10,2,5)
かかった時間(途中経過): 165 milliseconds

p=5, n=1, q=5, q^2+q+1=31, q+1=6, q^3+1=126, Number of MagicCircles = 5
(1,2,5,4,6,13), (1,2,7,4,12,5), (1,3,2,7,8,10), (1,3,6,2,5,14), (1,7,3,2,4,14)
かかった時間(途中経過): 166 milliseconds

p=7, n=1, q=7, q^2+q+1=57, q+1=8, q^3+1=344, Number of MagicCircles = 6
(1,2,10,19,4,7,9,5), (1,3,5,11,2,12,17,6), (1,3,8,2,16,7,15,5), (1,4,2,10,18,3,11,8), (1,4,22,7,3,6,2,12), (1,6,12,4,21,3,2,8)
かかった時間(途中経過): 167 milliseconds

p=2, n=3, q=8, q^2+q+1=73, q+1=9, q^3+1=513, Number of MagicCircles = 4
(1,2,4,8,16,5,18,9,10), (1,4,7,6,3,28,2,8,14), (1,6,4,24,13,3,2,12,8), (1,11,8,6,4,3,2,22,16)
かかった時間(途中経過): 173 milliseconds

p=3, n=2, q=9, q^2+q+1=91, q+1=10, q^3+